In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/hope-task1/valid.csv
/kaggle/input/hope-task1/train.csv
/kaggle/input/hope-task1/test.csv


In [2]:
!pip install tweet-preprocessor


In [3]:
import pandas as pd
import re, string
import numpy as np

import preprocessor as p

In [4]:
import random, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [5]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [6]:
path_train = "/kaggle/input/hope-task1/train.csv"
path_val = "/kaggle/input/hope-task1/valid.csv"
path_test = "/kaggle/input/hope-task1/test.csv"

# Read dataset

In [7]:
import string,re
def preprocessing_text(text):
    text = p.clean(text) # This method change the preprocessing function from simple version to enhanced version
    
    text = text.strip()
    text = text.translate(text.maketrans('', '', string.punctuation.replace("_","")))
    text = re.sub('\\s+',' ',text).strip()
    return text
#print(preprocessing_text(train_texts[0]))

In [8]:
import pandas as pd

def convert_label(text):
    if text == "hs":
        return 1
    else:
        return 0

def read_and_preprocessing(path_data):
    df = pd.read_csv(path_data)
    df["category"] = df["category"].apply(convert_label)
    x_input = df["text"].apply(preprocessing_text).tolist()
    y_output = df["category"].tolist()
    ids = df["id"].tolist()
    return x_input,y_output,ids

train_texts, train_labels,train_ids = read_and_preprocessing(path_train)
valid_texts,valid_labels,valid_ids = read_and_preprocessing(path_val)
print(len(train_texts),len(train_labels))
print(len(valid_texts),len(valid_labels))

1400 1400
200 200


In [9]:
df_train = pd.DataFrame(list(zip(train_texts, train_labels)),
               columns =['x_data', 'y_output'])
df_train.head()

,x_data,y_output
0,Si quieres saber lo diverso e inclusivo que es...,0
1,Y ya esta ya me he liberadoEs que las injustic...,1
2,me molesta mucho el discurso ese en favor del ...,0
3,Yo te lo explicoNo se pueden cortar las vas pb...,1
4,No somos opositores al movimiento por el contr...,0


# Build Filter Classifier

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#Try different models here:
model_name = "microsoft/deberta-v3-base" 
model_name2 = ""
model_name3 = ""

bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

In [11]:
max_length = 512
train_encodings = tokenizer(train_texts, truncation=True, max_length=max_length, padding=True)

In [12]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)

In [13]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # batch size per device during training
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_total_limit = 1,
    report_to = "tensorboard"
)

trainer = Trainer(
    model=bert_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)
trainer.train()
# API: fac85ccacc3dffb183116aba932e6bcc08010443

2024-04-16 18:06:52.851517: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 18:06:52.851646: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 18:06:52.984772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

Step,Training Loss
100,0.603100
200,0.285300
300,0.130500
400,0.050800


TrainOutput(global_step=440, training_loss=0.24612653770230034, metrics={'train_runtime': 172.2625, 'train_samples_per_second': 81.271, 'train_steps_per_second': 2.554, 'total_flos': 647511474960000.0, 'train_loss': 0.24612653770230034, 'epoch': 10.0})

# Inference

In [14]:
def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation=True, max_length = max_length, padding=True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label

In [ ]:
y_pred = []

for review in valid_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred.append(a)

In [16]:
from sklearn.metrics import *

print("M_Pr: ", round(precision_score(valid_labels, y_pred, average='macro'),4))
print("M_Re: ", round(recall_score(valid_labels, y_pred, average='macro'),4))
print("M_F1: ", round(f1_score(valid_labels, y_pred, average='macro'),4))

print("W_Pr: ", round(precision_score(valid_labels, y_pred, average='weighted'),4))
print("W_Re: ", round(recall_score(valid_labels, y_pred, average='weighted'),4))
print("W_F1: ", round(f1_score(valid_labels, y_pred, average='weighted'),4))

print("acc:", round(accuracy_score(valid_labels, y_pred), 4))

M_Pr:  0.8201
M_Re:  0.82
M_F1:  0.82
W_Pr:  0.8201
W_Re:  0.82
W_F1:  0.82
acc: 0.82


# Submission for validation datasets

In [17]:
# def convert2category(y_pred):
#     y_label = []
#     for y in y_pred:
#         if y == 0:
#               y_label.append("nhs")
#         else:
#               y_label.append("hs")
#     return y_label

In [18]:
# name_sub = "predictions.csv"
# y_pred_label = convert2category(y_pred)
# df_sub = pd.DataFrame(list(zip(valid_ids, y_pred_label)),
#                columns =['id', 'category'])
# df_sub.to_csv(name_sub)
# df_sub.head()

# Submission for test datasets


In [19]:
def read_and_preprocessing_for_test(path_data):
    df = pd.read_csv(path_data)
    x_input = df["text"].apply(preprocessing_text).tolist()
    ids = df["id"].tolist()
    return x_input,ids

test_texts,test_ids = read_and_preprocessing_for_test(path_test)

def make_prediction(review,tokenizer,trainer):
    demo_input = preprocessing_text(review)
    demo_encodings = tokenizer([demo_input], truncation=True, max_length = max_length, padding=True)
    test_dataset = CustomDataset(demo_encodings, [0])
    predic_demo = trainer.predict(test_dataset)[0]
    predict_label = np.argmax(predic_demo, axis=1).flatten().tolist()[0]
    return predict_label


In [ ]:
y_pred_test = []

for review in test_texts:
    a = make_prediction(review,tokenizer,trainer)
    y_pred_test.append(a)

In [21]:
def convert2category(y_pred):
    y_label = []
    for y in y_pred:
        if y == 0:
              y_label.append("nhs")
        else:
              y_label.append("hs")
    return y_label

In [22]:
name_sub = "predictions.csv"
y_pred_label = convert2category(y_pred_test)
df_sub = pd.DataFrame(list(zip(test_ids, y_pred_label)),
               columns =['id', 'category'])
df_sub.to_csv(name_sub)
df_sub.head()

,id,category
0,1596901421516931072,nhs
1,1592726714735468544,nhs
2,1294982700865146880,nhs
3,1471000234348056576,nhs
4,1316785660016156672,nhs
